In [1]:
import os

import pandas as pd
import plotly.express as px

In [2]:
data_dir = "stress_test_results_5090"
os.chdir(data_dir)
os.listdir()

['baseline_results_run1.csv',
 'baseline_results_run2.csv',
 'baseline_results_run3.csv',
 'improved_results_run1.csv',
 'improved_results_run2.csv',
 'improved_results_run3.csv',
 'stress_test.py',
 'stress_test_metadata.csv']

In [3]:
def load_df(path):
    df = pd.read_csv(path)
    df.drop(columns=["Unnamed: 0"], inplace=True)
    return df


run = "run1"

improved_path = f"improved_results_{run}.csv"
baseline_path = f"baseline_results_{run}.csv"

df_improved = load_df(improved_path)
df_improved["version"] = "improved"

df_baseline = load_df(baseline_path)
df_baseline["version"] = "baseline"

# df = df_improved
df = pd.concat([df_improved, df_baseline], ignore_index=True)
df

,dtype,quant_type,blocksize,tensor_shape,quantize_dequantize_latency,quantize_latency,dequantize_latency,version
0,torch.float32,fp4,64,1024,0.015702,0.010598,0.003007,improved
1,torch.float32,fp4,64,2048,0.018287,0.013075,0.003388,improved
2,torch.float32,fp4,64,4096,0.029649,0.022338,0.005022,improved
3,torch.float32,fp4,64,8192,0.084350,0.059975,0.026488,improved
4,torch.float32,fp4,64,16384,0.300955,0.208527,0.092865,improved
...,...,...,...,...,...,...,...,...
415,torch.bfloat16,nf4,4096,1024,0.014979,0.009746,0.003112,baseline
416,torch.bfloat16,nf4,4096,2048,0.016262,0.010629,0.003444,baseline
417,torch.bfloat16,nf4,4096,4096,0.021122,0.014035,0.004827,baseline
418,torch.bfloat16,nf4,4096,8192,0.045369,0.030330,0.014679,baseline


In [4]:
# # make a 3d plot with x as blocksize, y as tensor_shape, z as mean_quant_time, color by version
# fig = px.scatter_3d(
#     df, x="blocksize", y="tensor_shape", z="mean_quant_time", color="version",
#     title="Stress Test Results",
#     labels={"blocksize": "Block Size", "tensor_shape": "Tensor Shape", "mean_quant_time": "Mean Quant Time"},
# )

# # set the actual x and y axis labels
# fig.update_layout(
#     scene=dict(
#         xaxis_title="Block Size",
#         yaxis_title="Tensor Shape",
#         zaxis_title="Mean Quant Time (s)",
#     ),
#     legend_title_text="Version"
# )
# # set the tick labels to be the exact values
# fig.update_layout(
#     scene=dict(
#         xaxis=dict(tickmode='array', tickvals=df.blocksize.unique()),
#         yaxis=dict(tickmode='array', tickvals=df.tensor_shape.unique()),
#     )
# )

# # set xaxis and yaxis to be log scale
# fig.update_layout(
#     scene=dict(
#         xaxis=dict(type='log'),
#         yaxis=dict(type='log'),
#     )
# )

# fig.update_traces(marker=dict(size=5))
# fig.show()


In [5]:
def plot(df, algorithm, dtype):
    df = df[df.dtype == dtype]
    param = algorithm
    fig = px.line(
        df,
        x="blocksize",
        y=param,
        color="version",
        facet_row="tensor_shape",
        facet_col="quant_type",
        title=f"{algorithm} by Block Size [100 iterations total latency per blocksize][{dtype}]",
        labels={param: "", "blocksize": "Block Size", "tensor_shape": "dim"},
    )

    fig.add_annotation(
        text=f"Total {algorithm} Time (s)",
        xref="paper",
        yref="paper",
        x=-0.125,
        y=0.5,
        showarrow=False,
        textangle=-90,
        font=dict(size=14),
    )

    # set discrete x-tick labels using unique blocksize values
    blocksize_values = sorted(df["blocksize"].unique())
    for axis in fig.layout:
        if axis.startswith("xaxis"):
            fig.layout[axis].type = "category"
            fig.layout[axis].tickmode = "array"
            fig.layout[axis].tickvals = blocksize_values
            fig.layout[axis].ticktext = [str(val) for val in blocksize_values]
        # elif axis.startswith("yaxis"):
        # fig.layout[axis].type = "log"

    fig.update_yaxes(matches=None)
    fig.update_layout(width=1000, height=700)
    fig.show()


plot(df, "quantize_dequantize_latency", "torch.float32")

In [6]:
def plot(df, algorithm, dtype):
    param = algorithm
    df = df[df.dtype == dtype]

    # Define a custom color palette (modify as needed)
    custom_colors = ["#1f77b4", "#ff7f0e", "#2ca02c", "#d62728", "#9467bd"]

    fig = px.bar(
        df,
        x="blocksize",
        y=param,
        color="version",
        facet_row="tensor_shape",
        facet_col="quant_type",
        barmode="group",
        title=f"{algorithm} by Block Size [100 iterations total][{dtype}]",  # \n(lower is better)",
        labels={param: "", "blocksize": "Block Size", "tensor_shape": "dim"},
        color_discrete_sequence=custom_colors,
    )

    fig.add_annotation(
        text=f"Total {algorithm} Time (s)",
        xref="paper",
        yref="paper",
        x=-0.1,
        y=0.5,
        showarrow=False,
        textangle=-90,
        font=dict(size=14),
    )

    # set discrete x-tick labels using unique blocksize values
    blocksize_values = sorted(df["blocksize"].unique())
    for axis in fig.layout:
        if axis.startswith("xaxis"):
            fig.layout[axis].type = "category"
            fig.layout[axis].tickmode = "array"
            fig.layout[axis].tickvals = blocksize_values
            fig.layout[axis].ticktext = [str(val) for val in blocksize_values]
        elif axis.startswith("yaxis"):
            fig.layout[axis].type = "log"

    fig.update_yaxes(matches=None)
    fig.update_layout(width=1200, height=800)
    fig.show()


plot(df, "quantize_latency", "torch.float32")

In [7]:
# cols = [
#     "total_quantize_dequantize_time",
#     "total_quantize_dequantize_interleaved_time",
#     "total_quantize_time",
#     "total_dequantize_time",
# ]

# diff = (df_baseline[cols] - df_improved[cols]) / df_baseline[cols] * 100
# diff["blocksize"] = df_baseline["blocksize"]
# diff["tensor_shape"] = df_baseline["tensor_shape"]
# diff = diff[
#     [
#         "tensor_shape",
#         "blocksize",
#         "total_quantize_dequantize_time",
#         "total_quantize_dequantize_interleaved_time",
#         "total_quantize_time",
#         "total_dequantize_time",
#     ]
# ]
# diff.sort_values(by=["tensor_shape", "blocksize"])